In [7]:
# use conda enviroment "general" to run this script

import pysam
import glob
import gzip

from determine_PAS import determine_PAS
from does_the_read_contain_intron import does_the_read_contain_intron

In [8]:
file_directory = '/Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/realigned_BAM_files/'

all_BAM_files = glob.glob(file_directory + '*.bam')

print(len(all_BAM_files))

3281


In [9]:
output = []
i = 0

for BAM_file in all_BAM_files:
    
    i += 1
    
    dict = {}

    with pysam.AlignmentFile(BAM_file) as bam:
        
        for read in bam:
            
            if read.is_supplementary or read.is_secondary or read.is_unmapped:
                
                continue
            
            if read.mapping_quality < 20:
                
                continue
            
            
            
            #query_name = read.query_name
            query_sequence = read.query_sequence
            query_length = read.query_length
            alignment = read.get_aligned_pairs()
            
            
            
            PAS_in_reference = determine_PAS(query_sequence, alignment)
            contains_intron = does_the_read_contain_intron(read)
            
            
            
            code = f'{PAS_in_reference},{contains_intron}, {query_length}'
                # the f before a string denotes an f-string, which allows embedding expressions inside a string using curly braces {}
            
            if code in dict.keys():
                
                dict[code] += 1
                
            else:
                
                dict[code] = 1
    
    
    
    # BAM_file will look like: realigned_BAM_files/barcode12_____wild_type---200%_#1---100%_#2.bam
    BAM_file_name = BAM_file.split('/')[-1].split('.')[0]
    
    print('finish extraction from:', BAM_file_name)
    print(len(all_BAM_files) - i, 'BAM filels need to be processed')
    
    
    
    for code, read_counts in dict.items():
        
        output += [f"{BAM_file_name},{code},{read_counts}"]

finish extraction from: barcode04_____wild_type---100%_#2---10%_25%_#1
3280 BAM filels need to be processed
finish extraction from: barcode12_____wild_type---200%_#1---100%_#3
3279 BAM filels need to be processed
finish extraction from: barcode04_____wild_type---50%_#2---50%_#1
3278 BAM filels need to be processed
finish extraction from: barcode06_____wild_type---25%_#2---100%_#3
3277 BAM filels need to be processed
finish extraction from: barcode11_____wild_type---25%_#1---25%_#2
3276 BAM filels need to be processed
finish extraction from: barcode07_____UG_removed---10%_#3---100%_#3
3275 BAM filels need to be processed
finish extraction from: barcode04_____wild_type---10%_#1---50%_#1
3274 BAM filels need to be processed
finish extraction from: barcode01_____wild_type---0%_#1---0%_#1
3273 BAM filels need to be processed
finish extraction from: barcode11_____wild_type---100%_#3---100%_#1
3272 BAM filels need to be processed
finish extraction from: barcode12_____wild_type---10%_#3---100%

In [10]:
CSV_file = '../9_analyse results from minION with Flow Cell (R10.4.1)/for_final_analysis.csv.gz'

with gzip.open(CSV_file, 'wb') as file: # 'wb' will write in binary mode
    
    file.write('\n'.join(['insert,PAS_in_reference,contains_intron, read_length,read_counts'] + output).encode()) # encode() will convert string to bytes